In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import os
from djitellopy import Tello

# Create an FFmpeg input stream for the Tello's video stream
input_stream = (
    ffmpeg.input("udp://@0.0.0.0:11111", format="mpegts")
    .output("pipe:", format="rawvideo", pix_fmt="rgb24")
    .run_async(pipe_stdout=True)
)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

tello = Tello()
print("Connecting to Tello...")
tello.connect()
print("Connected. Starting video stream...")

tello.streamon()
# Try to load existing data, if any
file_name = "gesture_data.pkl"
if os.path.exists(file_name):
    with open(file_name, "rb") as f:
        data, labels = pickle.load(f)
else:
    data, labels = [], []

while True:

     # Read a frame from the FFmpeg input stream
    in_bytes = input_stream.stdout.read(960 * 720 * 3)  # Adjust the frame size as needed
    if not in_bytes:
        break

    # Convert the raw frame bytes to a NumPy array
    frame = np.frombuffer(in_bytes, np.uint8).reshape(720, 960, 3)
    
    frame_read = tello.get_frame_read()
    if frame_read.stopped:
        print("Stream stopped.")
        break

        # Convert the raw frame bytes to a NumPy array
    frame = np.frombuffer(in_bytes, np.uint8).reshape(720, 960, 3)

    # Display the frame
    cv2.imshow("Tello Frame", frame)


    results = hands.process(frame)
    

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_hands.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        landmarks = results.multi_hand_landmarks[0].landmark
        landmarks = np.array([[lmk.x, lmk.y, lmk.z] for lmk in landmarks]).flatten()


        key = cv2.waitKey(1)

        if key == ord("q"):
            break
        elif key == ord("1"):
            data.append(landmarks)
            labels.append("gesture_1")#lajk
        elif key == ord("2"):
            data.append(landmarks)
            labels.append("gesture_2")#ukazovak hore
        elif key == ord("3"):
            data.append(landmarks)
            labels.append("gesture_3")#rozostreta dlan
        elif key == ord("4"):
            data.append(landmarks)
            labels.append("gesture_4")#zovreta past stop
        elif key == ord("5"):
            data.append(landmarks)
            labels.append("gesture_5")#rock
        elif key == ord("6"):
            data.append(landmarks)
            labels.append("gesture_6")#doleukazovacik
        elif key == ord("7"):
            data.append(landmarks)
            labels.append("gesture_7")#ok
        #dalsie

# Save collected data
with open("gesture_data.pkl", "wb") as f:
    pickle.dump((data, labels), f)

print(labels)
tello.streamoff()
cv2.destroyAllWindows()


NameError: name 'ffmpeg' is not defined